In [4]:
import os
from dotenv import load_dotenv
from transformers import pipeline
import pandas as pd

In [2]:
load_dotenv()

True

In [8]:
year = int(os.getenv("YEAR"))
year

2020

In [9]:
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

In [10]:
classifier(['Aucun effort', "Très bon ensemble"])

[{'label': '1 star', 'score': 0.7614656686782837},
 {'label': '5 stars', 'score': 0.6455514430999756}]

In [11]:
df = pd.read_csv(f'./out/{year}/data_clean.csv')

In [12]:
df.shape

(7114, 5)

## Sentiment analysis

In [13]:
# Create list of dict like {'label': '5 stars', 'score': 0.7564778923988342} for each comment
# LONG TASK ! (45min)
labels = classifier(df['commentaire'].to_list())

In [14]:
# Create a dataframe from the list of dict
df_labels = pd.DataFrame.from_records(labels)
df_labels.shape

(7114, 2)

In [15]:
df_labels.head()

,label,score
0,5 stars,0.756478
1,4 stars,0.489843
2,3 stars,0.775267
3,4 stars,0.485216
4,4 stars,0.472740


### Clean the generated dataframe

In [16]:
# Only keep the number of stars in the label
df_labels['label'] = df_labels['label'].str.replace("\s.*", "", regex=True)
df_labels.head()

,label,score
0,5,0.756478
1,4,0.489843
2,3,0.775267
3,4,0.485216
4,4,0.472740


In [17]:
# Cast the label column to integer
df_labels['label'] = pd.to_numeric(df_labels['label'])
df_labels.dtypes

label      int64
score    float64
dtype: object

### Save labels

In [18]:
df_complete = pd.concat([df, df_labels], axis=1)
df_complete.head()

,eleve,classe,min,max,commentaire,label,score
0,17.80,14.28,8.30,17.80,"Un excellent début d'année, poursuivez ainsi !",5,0.756478
1,15.00,14.51,11.00,19.50,Bon ensemble.,4,0.489843
2,10.60,10.87,5.47,16.70,Résultats corrects mais le comportement en cla...,3,0.775267
3,15.50,14.24,6.00,19.00,Bon trimestre. Continuez ainsi !,4,0.485216
4,14.75,12.67,6.00,18.25,Un bon trimestre.,4,0.472740


### Clean labels

In [19]:
df_complete.describe()
# Mean score of .55 is correct

,classe,min,max,label,score
count,7114.000000,7114.000000,7114.000000,7114.000000,7114.000000
mean,12.197690,5.590322,17.806354,3.590104,0.560866
std,2.088382,3.550014,1.709161,1.006847,0.110026
min,6.070000,0.000000,11.000000,1.000000,0.228054
25%,10.910000,3.080000,16.800000,3.000000,0.488777
50%,12.130000,5.410000,18.000000,4.000000,0.538139
75%,13.340000,7.820000,19.080000,4.000000,0.618923
max,20.000000,20.000000,20.000000,5.000000,0.959330


In [20]:
# What is the upper score of the 5% worst labels
df_complete['score'].quantile(0.05)

0.4215182900428772

In [21]:
# Remove 5% worst labels
mask = df_complete['score'] > df_complete['score'].quantile(0.05) 
df_complete = df_complete[mask]

In [22]:
df_complete.to_csv(f'./out/{year}/data_clean_bert_labels.csv', index=False)

In [23]:
df_complete.shape

(6758, 7)